<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S105ocrCorrectionV06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# correction of ocr-generated text
- using correction dictionaries
- using rules


In [ ]:
import re, os, sys


In [ ]:
# core starts here - critical

!wget https://heibox.uni-heidelberg.de/f/c977e87cf2b244e6801b/?dl=1
!mv index.html?dl=1 KorpusARM.tgz


In [ ]:
!tar xvzf KorpusARM.tgz
!mkdir KorpusARM1
!mkdir KorpusARM1/stage01
# concatenating files
!cat korpusARM/hyFiktion/* >KorpusARM1/stage01/hyFiktion.txt
!cat korpusARM/hyNatur/* >KorpusARM1/stage01/hyNatur.txt
!cat korpusARM/hyRecht/* >KorpusARM1/stage01/hyRecht.txt
!mkdir KorpusARM1/stage02

# function for Armenian line breaks:

def correctLineBreaksHY(FName, FNameOut):
    FIn = open(FName, 'r')
    FOut = open(FNameOut, 'w')
    countHyphens = 0
    for SLine in FIn:
        SLine = SLine.strip()
        if SLine == '':
            FOut.write('\n\n')
            continue
        if SLine[-1] == '-':
            SLine2write = SLine[:-1]
            FOut.write(SLine2write)
            countHyphens +=1
            continue
        FOut.write(SLine + ' ')
    FOut.flush()
    print(str(countHyphens) + ' hyphens corrected')
    return

correctLineBreaksHY('KorpusARM1/stage01/hyFiktion.txt', 'KorpusARM1/stage02/hyFiktion.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyNatur.txt', 'KorpusARM1/stage02/hyNatur.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyRecht.txt', 'KorpusARM1/stage02/hyRecht.txt')


In [ ]:

!wc KorpusARM1/stage02/hyFiktion.txt
!wc KorpusARM1/stage02/hyNatur.txt
!wc KorpusARM1/stage02/hyRecht.txt

In [ ]:
# Dowloading the file with corrections
# !wget https://heibox.uni-heidelberg.de/f/14706c04a4024b2f937d/?dl=1
# without ճեր
!wget https://heibox.uni-heidelberg.de/f/4a24540473564788853d/?dl=1

!mv index.html?dl=1 Pilot-Corrections-all.tsv
!wc Pilot-Corrections-all.tsv



In [ ]:
# downloading wikipedia
### downloading Armenian Wikipedia
!wget https://heibox.uni-heidelberg.de/f/d1f866a61bd545318213/?dl=1
!mv index.html?dl=1 hywiki-20221101-pages-articles.txt.gz
!gunzip hywiki-20221101-pages-articles.txt.gz


In [ ]:
# the length of wikipedia
!wc hywiki-20221101-pages-articles.txt

  2446411  56531792 803098410 hywiki-20221101-pages-articles.txt


In [ ]:
# critical path function
def readCorrectionsFrq(colNumberOri, colNumberCorrect, colNumberFrq, SFIn, SFOut = None):
    LTWrongCorrect = []
    '''
    if type(LTWrongCorrect) == list:
        pass
    '''

    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.rstrip('\n')
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            SFrq = LLine[colNumberFrq]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]', f'{SFrq}')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect, SFrq in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\t{SFrq}\n')
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect


In [ ]:
# reading corrections for word forms, with the purpose of generalizing them
# goal: to display candidates for correction -- based on existing corrections (?)
LTWrongCorrectWordF, DWrongCorrectWordF = readCorrectionsFrq(1, 4, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-WordForm.tsv')
# LTWrongCorrectLemmaF, DWrongCorrectLemmaF = readCorrectionsFrq(3, 6, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-Lemma.tsv')
print(LTWrongCorrectWordF)
# print(LTWrongCorrectLemmaF)
# ինձ|ինչ
# առջև|առջևից
# ինչ|ինձ
# ինչ|ես
# գիտենալ|իմանալ


In [ ]:
print(len(DWrongCorrectWordF))
# print(len(DWrongCorrectLemmaF))


171


In [ ]:
for key, value in sorted(DWrongCorrectWordF.items()):
    print(f'{key}\t{value}')

This section applies corrections to an Armenian texts and records which words have been corrected

In [ ]:
# critical path function...
# function for Armenian tokenization
def tokenizeTextHY(SFIn):
    LLParagraphs = []
    with open(SFIn, 'r') as FIn:
        countpara = 0
        for SLine in FIn:
            countpara += 1
            if countpara % 100000 == 0: print(countpara)
            SLine = SLine.strip()
            if SLine == '': continue
            LLine = re.split('([ ,\.\:։;\'\"\(\)\-\–\!\?\{\}\t\«\»]+)', SLine)
            # if LLine == '': continue
            if LLine: LLParagraphs.append(LLine)
    return LLParagraphs



In [ ]:
!head --lines=40 /content/Parfum_Armenian.txt

## Critical path section
### Wikipedia

In [ ]:
try:
    del LLParaWiki
except:
    pass

In [ ]:
LLParaWiki = tokenizeTextHY('/content/hywiki-20221101-pages-articles.txt')

In [ ]:
print(LLParaWiki[1])

['Հայաստան', ' , ', 'պաշտոնական', ' ', 'անվանումը՝', ' ', 'Հայաստանի', ' ', 'Հանրապետություն', ', ', 'պետություն', ' ', 'Առաջավոր', ' ', 'Ասիայում՝', ' ', 'Հայկական', ' ', 'լեռնաշխարհի', ' ', 'հյուսիսարևելյան', ' ', 'մասում', '։ ', 'Քաղաքական', ' ', 'և', ' ', 'մշակութային', ' ', 'իմաստով', ', ', 'սակայն', ', ', 'գտնվում', ' ', 'է', ' ', 'հարավարևելյան', ' ', 'Եվրոպայի', ' ', 'Կովկասյան', ' ', 'տարածաշրջանում', '։ ', 'Հյուսիսում', ' ', 'սահմանակցում', ' ', 'է', ' ', 'Վրաստանին', ', ', 'արևելքում՝', ' ', 'Ադրբեջանին', ', ', 'հարավում՝', ' ', 'Իրանին', ', ', 'իսկ', ' ', 'արևմուտքում՝', ' ', 'Թուրքիային', '։ ', 'Հարավարևելյան', ' ', 'կողմում', ' ', 'Բերձորի', ' ', 'միջանցքով', ' ', 'կապվում', ' ', 'է', ' ', 'Արցախի', ' ', 'Հանրապետությանը', ', ', 'իսկ', ' ', 'հարավ', '-', 'արևմուտքում', ' ', 'Ադրբեջանի', ' ', 'էքսկլավ', ' ', 'Նախիջևանի', ' ', 'Ինքնավար', ' ', 'Հանրապետությունն', ' ', 'է', '։ ', 'Այժմյան', ' ', 'ՀՀ', '-', 'ն', ' ', 'զբաղեցնում', ' ', 'է', ' ', 'պատմական', ' ', 'Հայաստանի', 

In [ ]:
len(LLParaWiki[1])

137

In [ ]:
len(LLParaWiki)

2153019

In [ ]:
def llPara2dict(LLParagraphs):
    DFreq = {}
    p=0
    for LPara in LLParagraphs:
        p+=1
        if p%200000 == 0:
            print(p)
        # if LPara == [] or LPara == ['']: continue
        if LPara == []: continue
        # FOut.write(str(LPara) + '\n')
        i = 0 # counting words
        for el in LPara:
            i+=1 # index of next word
            try:
                DFreq[el] += 1
            except:
                DFreq[el] = 1
    return DFreq


In [ ]:
DWiki = llPara2dict(LLParaWiki)
print(len(DWiki))

200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2071984


In [ ]:
len(DWiki)

2071984

In [ ]:
FOut = open('hywiki-frqDict.txt', 'w')
for key, val in sorted(DWiki.items(), key=lambda item: item[1], reverse=True):
    FOut.write(f'{key}\t{val}\n')

In [ ]:
!wc hywiki-frqDict.txt

 2071974  4211029 42482907 hywiki-frqDict.txt


In [ ]:
!head --lines=40 hywiki-frqDict.txt

## Discover candidate rewriting rules systematically

In [ ]:
print(len(LTWrongCorrectWordF))

192


In [ ]:
for SWrong, SCorrect, Frq in LTWrongCorrectWordF:
    print(SWrong, SCorrect)

In [ ]:
def getPrefInfSuf(wrd1, wrd2):
    try:
        cpref = os.path.commonprefix([wrd1, wrd2])
        drw1 = wrd1[::-1]
        drw2 = wrd2[::-1]
        ffusc = os.path.commonprefix([drw1, drw2])
        csuff = ffusc[::-1]
    except:
        sys.stderr.write('error finding pref- and suffix')
        cpref = None
        csuff = None

    try:
        wrd1minpref = wrd1.removeprefix(cpref)
        wrd2minpref = wrd2.removeprefix(cpref)
        wrd1centre = wrd1minpref.removesuffix(csuff)
        wrd2centre = wrd2minpref.removesuffix(csuff)
    except:
        sys.stderr.write('error finding centre 1 and 2')
        wrd1centre = None
        wrd2centre = None

    return cpref, wrd1centre, wrd2centre, csuff


P12, I1, I2, S12 = getPrefInfSuf('[перепливи]', '[перелови]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[розгубився]', '[розгубивсь]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[вловив]', '[зловив]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[переходити]', '[перешкодити]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[переходити]', '[перешкоджати]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[ходити]', '[перешкоджати]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)




P12, I1, I2, S12 = getPrefInfSuf('[մերճակա]', '[մերձակա]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[առջն]', '[առջև]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[թեթնություն]', '[թեթևություն]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[Եթենա]', '[եթե նա]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[ննա]', '[նա]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)

P12, I1, I2, S12 = getPrefInfSuf('[ճեռքերն]', '[ձեռքից]')
print(P12, ' | ', I1, ' > ', I2, ' | ', S12)




[пере  |  пли  >  ло  |  ви]
[розгубивс  |  я  >  ь  |  ]
[  |  в  >  з  |  ловив]
[пере  |  х  >  шк  |  одити]
[пере  |  ходи  >  шкоджа  |  ти]
[  |  ходи  >  перешкоджа  |  ти]
[մեր  |  ճ  >  ձ  |  ակա]
[առջ  |  ն  >  և  |  ]
[թեթ  |  ն  >  և  |  ություն]
[  |  Եթե  >  եթե   |  նա]
[ն  |    >  ա]  |  նա]
[  |  ճեռքերն  >  ձեռքից  |  ]


In [ ]:
def createSufList(a):
    ''' odyty] --> odyty], odyty, odyt, ody, od, o, "" '''
    LSuf = []
    for j in range(1,len(a)+1):
        prefix = a[:j]
        LSuf.append(prefix)
        # print(j, len(prefix), LSuf)
    LSuf.insert(0, '')
    return LSuf

createSufList('odyty]')

['', 'o', 'od', 'ody', 'odyt', 'odyty', 'odyty]']

In [ ]:
createSufList('ություն]')

['', 'ո', 'ու', 'ութ', 'ությ', 'ությո', 'ությու', 'ություն', 'ություն]']

In [ ]:
def createPrefList(s):
    LPref = [s[-i:] for i in range(1, len(s) + 1)]
    LPref.insert(0, '')
    # print(LPref)
    return LPref

createPrefList('[pere')

['', 'e', 're', 'ere', 'pere', '[pere']

In [ ]:
createPrefList('[թեթ')

['', 'թ', 'եթ', 'թեթ', '[թեթ']

### we create the list of potential rewrite rules


In [ ]:
def twoWords2listOfRules(W1, W2):
    DTRules = {}
    DTRulesLen = {}
    LTRules = []
    P12, I1, I2, S12 = getPrefInfSuf(W1, W2)
    LPref12 = createPrefList(P12)
    LSuf12 = createSufList(S12)

    for pref in LPref12:
        for suf in LSuf12:
            SlhsRule = pref + I1 + suf
            SrhsRule = pref + I2 + suf
            TRules = (SlhsRule, SrhsRule)
            LTRules.append(TRules)
            try:
                DTRules[TRules] += 1
            except:
                DTRules[TRules] = 1

            try:
                DTRulesLen[TRules] = len(SlhsRule)
            except:
                continue
    return LTRules, DTRules, DTRulesLen

def printFrqTDict(DTFrq, FOut = None):
    for key, val in sorted(DTFrq.items(), key=lambda x:x[1], reverse=True):
        LHS, RHS = key
        if FOut:
            FOut.write(f'{LHS}, {RHS}, {str(val)}\n')
        else:
            print(LHS, RHS, str(val))
    if FOut:
        FOut.flush()

def printTList(LTuples, FOut = None):
    for el in LTuples:
        LHS, RHS = el
        if FOut:
            FOut.write(f'{LHS}, {RHS}\n')
        else:
            print(LHS, RHS)
    if FOut:
        FOut.flush()

LTRules, DTRules, DTRulesLen = twoWords2listOfRules('[перепливи]', '[перелови]')
FOut = open('printFrqTDict-test.txt', 'w')
printFrqTDict(DTRules, FOut)
FOut0 = open('printFrqTDictLen-test.txt', 'w')
printFrqTDict(DTRulesLen, FOut0)
FOut1 = open('printTList-test.txt', 'w')
printTList(LTRules, FOut1)

In [ ]:
LTRules, DTRules, DTRulesLen = twoWords2listOfRules('[թեթնություն]', '[թեթևություն]')
FOut0 = open('printTRulesLen-test-hy.txt', 'w')
printFrqTDict(DTRulesLen, FOut0)


In [ ]:
def findLongestMatch(SInput, DTRulesLength):
    SOutput = None
    for key, val in sorted(DTRulesLength.items(), key=lambda x:x[1], reverse=True):
        LHS, RHS = key
        if LHS in SInput:
            SOutput = SInput.replace(LHS, RHS, 1)
            break
    return SOutput

SOutput1 = findLongestMatch('[перепливи]', DTRulesLen)
if SOutput1: print(SOutput1)

SOutput1 = findLongestMatch('[перешкоджав]', DTRulesLen)
if SOutput1: print(SOutput1)

SOutput1 = findLongestMatch('[проходжав]', DTRulesLen)
if SOutput1: print(SOutput1)


[перелови]
